# Gestion des données tabulaires

Les données tabulaires (ensemble de données regroupées par colonnes et par lignes) sont universelles et omniprésentes. On les retrouve aussi bien dans nos listes de courses, nos listes de taches, la gestion comptable et le suivi des processus.

Des outils dédiés leurs sont consacrés (les tableurs) et elles sont à la base également de solutions élaborées comme par exemple pour le stockage (les 'tables' des bases de données relationnelles) et pour le traitement (les 'dataFrame' Pandas) de gros volumes de données.

Pourtant, malgré cette importante popularité, il n'existe que très peu d'outils permettant de comprendre, d'analyser et d'exploiter la structuration de ce type de données.

L'objet de cet article est donc de vous présenter les apports de ce type d'outil (les liens renvoient vers un [exemple d'implémentation](#2---Exemple-d'outil) en fin d'article).

## Que trouve-t-on dans un tableau ?

La première structure identifiable est évidente : les lignes et les colonnes !
Néanmoins, ces deux notions ne sont pas équivalentes, les colonnes représentent la 'sémantique' des données et les lignes représentent les objets rangés suivant cette structuration définie par les colonnes.

Si l'on observe maintenant comment sont utilisés les tableaux, on peut identifier trois usages principaux :
- la classification : Il s'agit de regrouper les objets par catégories afin de pouvoir par exemple en faire une exploitation statistique,
- le croisement : Il consiste à représenter toutes les combinaisons entre deux paramètres comme par exemple dans les représentations matricielles
- la caractérisation : Elle correspond à la documentation de propriétés définies.

> *Prenons un exemple :*
>
>|id|produit|aliment	|quantité	|prix|validité|disponibilité|
|:------:|:------:|:------:|:------:|:------:|:------:|:------:|
|11|pomme|fruit	|sachet 1 kg	|1	|du 1/7/2022 au 31/12/2022|oui|
|12|pomme|fruit	|carton 10 kg	|9|du 1/7/2022 au 31/12/2022|oui|
|13|orange|fruit	|sachet 1 kg|2|du 1/7/2022 au 31/12/2022|fin 2022|
|14|orange|fruit	|carton 10 kg	|18	|du 1/7/2022 au 31/12/2022|fin 2022|
|15|piment|légume	|sachet 1 kg	|1.5|du 1/7/2022 au 31/12/2022|oui|
|16|piment|légume	|carton 10 kg|13|du 1/7/2022 au 31/12/2022|fin 2022|
|17|banane|fruit	|sachet 1 kg	|0.5|du 1/7/2022 au 31/12/2022|oui|
|18|banane|fruit	|carton 10 kg	|4|du 1/7/2022 au 31/12/2022|oui|
>
>*Il s'agit d'une liste de prix de différents aliments en fonction d'un conditionnement pour l'année 2022.*
>
> *On retrouve ici :*
> - *la classification : entre les produits et les aliments,* 
> - *le croisement : entre les produits et les quantités,*
> - *la caractérisation : la disponibilité du produit*

On peut noter que les deux usages de classification et de croisement conduisent à dupliquer fortement les informations avec les risques d'erreurs associés.

## Analyse des colonnes

Les relations entre deux colonnes peuvent être qualifiées et mesurées en fonction de ces trois usages. 
Ceci permet par exemple :
- de contrôler qu'une exigence est bien respectée,
- de retrouver les données qui ne respectent pas l'exigence 

> *Dans l'exemple précédent, on vérifie que le champ 'aliment' est bien 'derivé' du champ 'produit' ([voir partie 2](#lien-'aliment'-'produit')).*
>
> *Si l'on introduit une association ('banane', 'légume') au lieu de ('banane', 'fruit'), l'exigence de couplage n'est plus respectée [(voir partie 2)](#erreur-de-saisie). Les données incohérentes peuvent alors être explicitée [(voir partie 2)](#détection-des-erreurs).*


## Analyse globale

L'analyse de l'ensemble des liens entre champs permet de représenter la structure globale du tableau.    
Une première représentation exploite le lien de classification. Elle se traduit par un arbre de dépendance entre colonnes.     
Une deuxième représentation qualifie chacun des champs en fonction de son rôle dans la structure globale.

> *La première représentation appliquée à l'exemple se traduit par l'arborescence suivante ([voir partie 2](#représentation-globale)) :*
> ```python
> -1: root (8)    
>      0 : id (8)    
>            1 : produit (4)    
>                  2 : aliment (2)    
>            3 : quantite (2)
>            4 : prix (8)
>            6 : disponibilite (2)
>      5 : validite (1)
> ```
> *La racine de cette arborescence (root) représente la liste des lignes du tableaux.*    
> *Le champ 'aliment' est bien 'dérivé' du champ 'produit'.*    
> *la valeur entre parenthèse représente le nombre de valeurs différentes (les valeurs les plus faibles sont situées dans les feuilles de l'arbre).*
>     
> *La deuxième représentation est la suivante ([voir partie 2](#représentation-globale)) :*
> ```python
> {'id': 'variable',
> 'produit': 'primary',
> 'aliment': 'secondary',
> 'quantite': 'primary',
> 'prix': 'coupled',
> 'validite': 'unique',
> 'disponibilite': 'variable'}
> ```
> *On distingue les catégories suivantes :*
> - *primary: qualifie les champs 'croisés' ('produit' et 'quantite')*
> - *secondary: représente les champs 'dérivés' des champs 'primary' ('aliment')*
> - *coupled: est une catégorie spécique indiquant que chaque valeur de ce champ est associée à une unique valeur d'un autre champ (chaque valeur du champ 'prix' correspond à une valeur du champ 'id')*
> - *unique: pour les champs composés d'une seule valeur ('validite')
> - *variable: pour les autres champs qui ne disposent pas de relations particulières ('id', 'disponibilite')*

## Export vers des outils d'analyse
Certains outils exploitent nativement ces structures de données. C'est le cas par exemple de l'outil Xarray qui représente les données à la fois sous forme matricielle et sous forme de dépendances avec chacuns des axes de la matrice.

> *L'export des données de l'exemple vers Xarray produit la structure suivante :*
> ```python
> <xarray.DataArray 'prix' (produit: 4, quantite: 2)>
> array([[20, 2],
>       [15, 1.5],
>       [5, 0.5],
>       [10, 1]], dtype=object)
>Coordinates:
>  * produit       (produit) object 'orange' 'piment' 'banane' 'pomme'
>  * quantite      (quantite) object '10 kg' '1 kg'
>    aliment       (produit) object 'fruit' 'legume' 'fruit' 'fruit'
>Attributes:
>    validite:    2022
> ```
> *Les données dans la matrice sont celles liées au champ 'prix', on aurait pu également choisir celles d'un autre champ 'variable' ('id' ou 'disponibilite')*
> *Les deux dimensions de la matrice correspondent aux deux champs 'produit' et 'quantite'.*
> *Le champ 'aliment' est associé au champ 'produit' au niveau de la structure 'coordinates'.*
> *Le champ 'annee' est unique et se trouve donc associé à la structure 'attributes'.*

## Optimisation des formats d'échange
Le format privilégié d'échange des données tabulaires est le format csv qui est l'image exacte d'un tableau (chaque ligne du tableau est une ligne du fichier csv).   
Ce format n'est pas optimal dans la mesure où les données dupliquées du tableau se retrouve également dupliquées dans le fichier csv.   
    
La prise en compte des liens entre champs permet de supprimer toutes les duplications et de les remplacer par une codification optimisée. Le gain de volume est alors conséquent puisque la taille du fichier est divisée par 5 à 10 sur des jeux de données importants.



------------------

# 2 - Exemple d'outil

### lien 'aliment' 'produit'

In [ ]:
from observation import Ilist

tarif = Ilist.obj([['id', [11, 12, 13, 14, 15, 16, 17, 18]],
                   ['produit', ['pomme', 'pomme', 'orange', 'orange', 'piment', 'piment', 'banane', 'banane']],
                   ['aliment', ['fruit', 'fruit', 'fruit', 'fruit', 'legume', 'legume', 'fruit', 'fruit']],
                   ['quantite', ['sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg']],
                   ['prix', [1, 10, 2, 20, 1.5, 15, 0.5, 5]],
                   ['validite', ['du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022']],
                   ['disponibilite', ['oui', 'oui','fin 2022','fin 2022','oui','fin 2022','oui','oui']]])

id, produit, aliment, quantite, prix, validite, dispo = tarif.lindex

aliment.isderived(produit)

### erreur de saisie

In [2]:
tarif_erreur = Ilist.obj([['id', [11, 12, 13, 14, 15, 16, 17, 18]],
                   ['produit', ['pomme', 'pomme', 'orange', 'orange', 'piment', 'piment', 'banane', 'banane']],
                   ['aliment', ['fruit', 'fruit', 'fruit', 'fruit', 'legume', 'legume', 'legume', 'fruit']],
                   ['quantite', ['sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg']],
                   ['prix', [1, 10, 2, 20, 1.5, 15, 0.5, 5]],
                   ['validite', ['du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022']],
                   ['disponibilite', ['oui', 'oui','fin 2022','fin 2022','oui','fin 2022','oui','oui']]])

id_, produit_, aliment_, quantite_, prix_, validite_, dispo_ = tarif_erreur.lindex

aliment_.isderived(produit_)

False

### détection des erreurs

In [3]:
produit_.coupling(aliment_)
tarif_erreur.getduplicates(['produit'], indexview=['produit', 'aliment'] )

[('banane', 'legume'), ('banane', 'fruit')]

### représentation globale

In [4]:
print(tarif.tree)
tarif.category

-1: root (8)
      0 : id (8)
            1 : produit (4)
                  2 : aliment (2)
            3 : quantite (2)
            4 : prix (8)
            6 : disponibilite (2)
      5 : annee (1)


{'id': 'variable',
 'produit': 'primary',
 'aliment': 'secondary',
 'quantite': 'primary',
 'prix': 'coupled',
 'annee': 'unique',
 'disponibilite': 'variable'}

### Export Xarray

In [5]:
tarif.to_xarray(varname='prix')

<xarray.DataArray 'Ilist' (produit: 4, quantite: 2)>
array([[0.5, 5],
       [2, 20],
       [1.5, 15],
       [1, 10]], dtype=object)
Coordinates:
  * produit       (produit) object 'banane' 'orange' 'piment' 'pomme'
    produit_row   (produit) int32 0 1 2 3
    produit_str   (produit) object 'banane' 'orange' 'piment' 'pomme'
  * quantite      (quantite) object '1 kg' '10 kg'
    quantite_row  (quantite) int32 0 1
    quantite_str  (quantite) object '1 kg' '10 kg'
    aliment       (produit) object 'fruit' 'fruit' 'legume' 'fruit'
Attributes:
    annee:    2022

In [6]:
tarif.to_xarray(varname='disponibilite')

<xarray.DataArray 'Ilist' (produit: 4, quantite: 2)>
array([['oui', 'oui'],
       ['non', 'non'],
       ['oui', 'non'],
       ['oui', 'oui']], dtype=object)
Coordinates:
  * produit       (produit) object 'banane' 'orange' 'piment' 'pomme'
    produit_row   (produit) int32 0 1 2 3
    produit_str   (produit) object 'banane' 'orange' 'piment' 'pomme'
  * quantite      (quantite) object '1 kg' '10 kg'
    quantite_row  (quantite) int32 0 1
    quantite_str  (quantite) object '1 kg' '10 kg'
    aliment       (produit) object 'fruit' 'fruit' 'legume' 'fruit'
Attributes:
    annee:    2022

### Format d'échange

In [9]:
from pprint import pprint
tarif.setcanonorder()
taille_csv = len(tarif.json(modecodec='full', encoded=True))
taille_opt = len(tarif.json(modecodec='optimize', encoded=True))
taille_bin = len(tarif.json(modecodec='optimize', encoded=True, encode_format='cbor'))
print('taille : ', taille_csv, taille_opt, taille_bin, '\n')
print('\nmode équivalent csv :')
pprint(tarif.json(modecodec='full'))
print('\nmode optimisé :')
pprint(tarif.json(modecodec='optimize'))

taille :  489 313 169 


mode équivalent csv :
[['produit',
  ['banane',
   'banane',
   'orange',
   'orange',
   'piment',
   'piment',
   'pomme',
   'pomme']],
 ['quantite',
  ['1 kg', '10 kg', '1 kg', '10 kg', '1 kg', '10 kg', '1 kg', '10 kg']],
 ['aliment',
  ['fruit', 'fruit', 'fruit', 'fruit', 'legume', 'legume', 'fruit', 'fruit']],
 ['annee', [2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022]],
 ['id', [17, 18, 13, 14, 15, 16, 11, 12]],
 ['disponibilite', ['oui', 'oui', 'non', 'non', 'oui', 'non', 'oui', 'oui']],
 ['prix', [0.5, 5, 2, 20, 1.5, 15, 1, 10]]]

mode optimisé :
[['produit', ['banane', 'orange', 'piment', 'pomme']],
 ['quantite', ['1 kg', '10 kg']],
 ['aliment', ['fruit', 'legume'], [0, [0, 0, 1, 0]]],
 ['annee', [2022]],
 ['id', [17, 18, 13, 14, 15, 16, 11, 12]],
 ['disponibilite', ['non', 'oui'], [4, [1, 1, 0, 0, 1, 0, 1, 1]]],
 ['prix', [0.5, 5, 2, 20, 1.5, 15, 1, 10], 4]]
